In [1]:
from Tools import *

In [2]:
# Задаем значение для генератора случайных чисел
seed_value = 23
np.random.seed(seed_value)

In [3]:
## creating dataset
c = 12052
X1 = pd.DataFrame()
X1['var_1'] = np.random.rand(c)
X1['var_2'] = np.random.rand(c)
X1['var_3'] = np.random.rand(c)
X1['var_4'] = np.random.rand(c)
X1['var_5'] = np.random.rand(c)
X1['var_6'] = np.random.rand(c)
X1['var_7'] = np.random.rand(c)
X1['var_8'] = np.random.rand(c)
X1['var_9'] = np.random.rand(c)
X1['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

## creating dependence y ~ x1 + 3x2 - x1^(1.5) + noise
y1 = X1['var_1'] + 3 * X1['var_2'] - np.power(X1['var_3'], 1.5) + np.random.rand(c)

X2 = pd.DataFrame()
X2['var_1'] = np.random.rand(c)
X2['var_2'] = np.random.rand(c)
X2['var_3'] = np.random.rand(c)
X2['var_4'] = np.random.rand(c)
X2['var_5'] = np.random.rand(c)
X2['var_6'] = np.random.rand(c)
X2['var_7'] = np.random.rand(c)
X2['var_8'] = np.random.rand(c)
X2['var_9'] = np.random.rand(c)
X2['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

y2 = X2['var_1'] + 3 * X2['var_2'] - np.power(X2['var_3'], 1.5) + np.random.rand(c)

X3 = pd.DataFrame()
X3['var_1'] = np.random.rand(c)
X3['var_2'] = np.random.rand(c)
X3['var_3'] = np.random.rand(c)
X3['var_4'] = np.random.rand(c)
X3['var_5'] = np.random.rand(c)
X3['var_6'] = np.random.rand(c)
X3['var_7'] = np.random.rand(c)
X3['var_8'] = np.random.rand(c)
X3['var_9'] = np.random.rand(c)
X3['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

y3 = X3['var_1'] + 3 * X3['var_2'] - np.power(X3['var_3'], 1.5) + np.random.rand(c)

X = pd.concat([X1, X2, X3])
y = pd.concat([y1, y2, y3])
X.shape

(36156, 10)

In [4]:
# creating cross validator
cv_datetime = DateTimeSeriesSplit(window=3000, n_splits=3, test_size=300, margin=0)
group_dt = X['date']

## get parametres from train and test for each fold
for fold, (train_idx, val_idx) in enumerate(cv_datetime.split(X, groups=group_dt), 1):
    print(fold)
    train, test = X.iloc[train_idx], X.iloc[val_idx]
    print(f'треин мин {train.date.min()} треин макс {train.date.max()} shape {train.shape}')
    print(f'тест мин {test.date.min()} тест макс {test.date.max()} shape {test.shape}')

1
треин мин 2012-04-27 00:00:00 треин макс 2020-07-13 00:00:00 shape (9000, 11)
тест мин 2020-07-14 00:00:00 тест макс 2021-05-09 00:00:00 shape (900, 11)
2
треин мин 2013-02-21 00:00:00 треин макс 2021-05-09 00:00:00 shape (9000, 11)
тест мин 2021-05-10 00:00:00 тест макс 2022-03-05 00:00:00 shape (900, 11)
3
треин мин 2013-12-18 00:00:00 треин макс 2022-03-05 00:00:00 shape (9000, 11)
тест мин 2022-03-06 00:00:00 тест макс 2022-12-30 00:00:00 shape (900, 11)


In [5]:
# create model for selector
from lightgbm import LGBMRegressor
model = LGBMRegressor(max_depth=3, objective='MAPE', verbosity = -1)

# create list from which we will take vars for model creating
list_of_vars = list(X.columns)
list_of_vars.remove('date')
list_of_vars.remove('index_time')

In [6]:
# import metrics for var selection
from sklearn.metrics import mean_absolute_percentage_error as MAPE

# create selector
selector1 = Kraken(model, cv_datetime, MAPE, 'exp1')

In [7]:
# get rank dict from vars
selector1.get_rank_dict(X, y, list_of_vars, group_dt)

# get ranked dict in descending order of abs shap value
selector1.rank_dict

Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "c:\Users\vitya\OneDrive\Рабочий стол\venvs\hardml\lib\site-packages\joblib\externals\loky\backend\context.py", line 245, in _count_physical_cores
    raise ValueError(


{'var_2': 1,
 'var_3': 2,
 'var_1': 3,
 'var_9': 4,
 'var_7': 5,
 'var_4': 6,
 'var_6': 7,
 'var_8': 8,
 'var_5': 9}

In [8]:
## get vars
selector1.get_vars(X, y, early_stopping_rounds = 10, group_dt = group_dt)

запуск первого шага
new var_for_add ! var_2
едем дальше
в итоге получили список ['var_2']
запуск первого шага
new var_for_add ! var_3
едем дальше
в итоге получили список ['var_2', 'var_3']
запуск первого шага
new var_for_add ! var_1
едем дальше
в итоге получили список ['var_2', 'var_3', 'var_1']
запуск первого шага
мы сошлись
['var_2', 'var_3', 'var_1']
0.247


['var_2', 'var_3', 'var_1']